In [14]:
!pip install ace_tools

In [15]:
import pandas as pd
from pathlib import Path

DATA_XLSX = Path("./GaitRite-GaitData/DCM_81_FWD_1.xlsx")          # ← update if needed
SHEET_NAME = 0                                 # or "Footfalls / ExportDetailed"


In [16]:
# Raw GAITRite column header as a python list
cols = [
    "Record#", "TestRecord#", "PtRecord#", "FootFallObject#", "DateTimeStamp",
    "HeelX", "HeelY", "ToeX", "ToeY",
    "LowestX", "HighestX", "LowestY", "HighestY",
    "FirstContactTime", "LastContactTime", "FootFlatTime",
    "BeginTime", "EndTime", "LRFoot",                   # 0=L,1=R
    "StepLength_cm", "StrideLength_cm", "BaseSupport_cm",
    "StepTime_s", "StrideTime_s", "SwingTime_s", "StanceTime_s",
    "SingleSupport_s", "DoubleSupport_s", "StrideVelocity", "RealFootFlag",
    "PassNumber", "ToeInOut_deg",
    "StepWidth_cm", "StrideWidth_cm",
    "HeelOn", "HeelOff", "MidOn", "MidOff", "ToeOn", "ToeOff",
    "Heel_off_on", "DS_Loading", "DS_Unloading"
]

df_raw = pd.read_excel(DATA_XLSX, sheet_name=SHEET_NAME,
                       names=cols, header=0)

# Parse date/time & map foot side
df_raw["DateTimeStamp"] = pd.to_datetime(df_raw["DateTimeStamp"])
df_raw["Foot"] = df_raw["LRFoot"].map({0: "L", 1: "R"})


In [17]:
# Replace obvious placeholder zeros with NaN to avoid skewing means
metrics_cols = [c for c in cols if c.endswith(("_cm", "_s"))]
df_raw[metrics_cols] = df_raw[metrics_cols].replace(0, pd.NA)

# Convert cm → m for spatial measures
for c in df_raw.filter(like="_cm").columns:
    df_raw[c.replace("_cm", "_m")] = df_raw[c] / 100.0


In [18]:
tidy_cols = [
    "FootFallObject#", "Foot", "DateTimeStamp",
    "StepLength_m", "StrideLength_m", "BaseSupport_m",
    "StepTime_s", "StrideTime_s", "SwingTime_s", "StanceTime_s",
    "StepWidth_m", "StrideWidth_m"
]
df_steps = df_raw[tidy_cols].copy()
df_steps = df_steps.sort_values("FootFallObject#").reset_index(drop=True)

display(df_steps)


,FootFallObject#,Foot,DateTimeStamp,StepLength_m,StrideLength_m,BaseSupport_m,StepTime_s,StrideTime_s,SwingTime_s,StanceTime_s,StepWidth_m,StrideWidth_m
0,1.0,R,2024-10-30 13:04:22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.85,<NA>,<NA>
1,2.0,L,2024-10-30 13:04:22,0.51985,<NA>,0.14387,0.625,<NA>,<NA>,0.842,0.54992,0.16347
2,3.0,R,2024-10-30 13:04:22,0.62306,1.14351,0.12913,0.617,1.242,0.392,0.833,0.63952,0.14696
3,4.0,L,2024-10-30 13:04:22,0.54614,1.16923,0.13346,0.608,1.225,0.383,0.85,0.56596,0.14873
4,5.0,R,2024-10-30 13:04:22,0.60863,1.15477,0.15081,0.617,1.225,0.392,0.85,0.62675,0.16507
5,6.0,L,2024-10-30 13:04:22,0.53437,1.14349,0.14778,0.608,1.225,0.375,<NA>,0.56355,0.16516
6,7.0,R,2024-10-30 13:04:22,0.63343,1.16847,<NA>,0.625,1.233,0.383,<NA>,0.65077,<NA>
7,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# =========================================================
#  Inspect HSMR / SKEL parameter file – quick structural peek
# =========================================================
from pathlib import Path
import numpy as np
import pprint, textwrap

FILE = Path("./HSMR-DCM-SKEL-params/HSMR-DCM_81_FWD_1.npy")  # <— update if you move it

def nice_shape(x):
    return getattr(x, "shape", None)

def inspect_skel(file_path, frames_to_show=2):
    print(f"🔍  Inspecting: {file_path.name}")
    print(f"📦  Size     : {file_path.stat().st_size/1024/1024:.2f} MB\n")

    obj = np.load(file_path, allow_pickle=True)

    # --------------------------------------------------------
    #  Case A – file is *dict-like* (either .npy or .npz)
    # --------------------------------------------------------
    if isinstance(obj, dict) or isinstance(obj, np.lib.npyio.NpzFile):
        keys = obj.keys() if isinstance(obj, dict) else obj.files
        print("📋  File stores ONE dict with keys:")
        for k in keys:
            print(f"   • {k:<12s} shape={nice_shape(obj[k])}")
        return

    # --------------------------------------------------------
    #  Case B – file is an ndarray of per-frame dicts
    # --------------------------------------------------------
    if isinstance(obj, np.ndarray) and obj.dtype == object:
        T = len(obj)
        print(f"🎬  Video-style file with {T} frames (array of dicts)\n")

        # show the first few frames
        for t in range(min(frames_to_show, T)):
            fr = obj[t]
            print(f"— Frame {t} —")
            if not isinstance(fr, dict):
                print("   ⚠️  frame is not a dict")
                continue
            for k, v in fr.items():
                if isinstance(v, np.ndarray):
                    print(f"   • {k:<12s} shape={v.shape}  dtype={v.dtype}")
                else:
                    # non-array items (e.g. bbx_cs is a list of arrays)
                    short = textwrap.shorten(repr(v), width=60, placeholder=" … ")
                    print(f"   • {k:<12s} {type(v).__name__}: {short}")

            # optional: dig a bit deeper for numeric keys
            if "poses" in fr:
                poses = fr["poses"].astype(float)
                print(f"     ↳ poses min/max/mean: {poses.min():.3f} / "
                      f"{poses.max():.3f} / {poses.mean():.3f}")
            if "betas" in fr:
                betas = fr["betas"].astype(float)
                print(f"     ↳ betas min/max:      {betas.min():.3f} / {betas.max():.3f}")
            print()
        return

    print(f"⚠️  Unrecognised file content type: {type(obj)}")

# ----------------------------------------------------------
inspect_skel(FILE, frames_to_show=3)   # <- adjust frames_to_show as desired


🔍  Inspecting: HSMR-DCM_81_FWD_1.npy
📦  Size     : 0.32 MB

🎬  Video-style file with 516 frames (array of dicts)

— Frame 0 —
   • patch_cam_t  shape=(1, 3)  dtype=float32
   • poses        shape=(1, 46)  dtype=float32
   • betas        shape=(1, 10)  dtype=float32
   • full_cam_t   shape=(1, 3)  dtype=float32
   • bbx_cs       list: [array([219.70868, 844.03754, 583.94104], dtype=float32)]
     ↳ poses min/max/mean: -2.368 / 1.075 / -0.006
     ↳ betas min/max:      -0.114 / 0.105

— Frame 1 —
   • patch_cam_t  shape=(1, 3)  dtype=float32
   • poses        shape=(1, 46)  dtype=float32
   • betas        shape=(1, 10)  dtype=float32
   • full_cam_t   shape=(1, 3)  dtype=float32
   • bbx_cs       list: [array([220.13959, 842.9415 , 584.4031 ], dtype=float32)]
     ↳ poses min/max/mean: -2.367 / 1.058 / -0.009
     ↳ betas min/max:      -0.110 / 0.108

— Frame 2 —
   • patch_cam_t  shape=(1, 3)  dtype=float32
   • poses        shape=(1, 46)  dtype=float32
   • betas        shape=(1, 10)  

In [20]:
# ------------------------------------------------------------
#  Deep-dive into a single frame of an HSMR / SKEL .npy video
# ------------------------------------------------------------
import numpy as np, pandas as pd, pprint, textwrap
from pathlib import Path
np.set_printoptions(suppress=True, linewidth=150)   # nicer printing

FILE = Path("./HSMR-DCM-SKEL-params/HSMR-DCM_81_FWD_1.npy")   # <— update
data = np.load(FILE, allow_pickle=True)

assert isinstance(data, np.ndarray) and data.dtype == object, "expected array-of-dicts"
T = len(data)
print(f"🗂  frames in file: {T}")

# ---------- pick the frame you want to inspect ---------------
idx = 0             # ← change to any 0 … T-1
fr  = data[idx]
print(f"\n🔍  Frame {idx}\n{'-'*40}")

for k, v in fr.items():
    if isinstance(v, np.ndarray):
        print(f"• {k}  shape={v.shape}  dtype={v.dtype}")
        print(v)                       # print full array
    else:
        # typical case: bbx_cs is a list of arrays
        short = textwrap.shorten(repr(v), width=120, placeholder=" … ")
        print(f"• {k}  {type(v).__name__}")
        print("  ", short)
    print()

# ---------- OPTIONAL: dump each key to disk ------------------
DUMP = False      # ← flip to True if you want files
OUT_DIR = FILE.parent / f"{FILE.stem}_frame{idx}"
if DUMP:
    OUT_DIR.mkdir(exist_ok=True)
    for k, v in fr.items():
        if isinstance(v, np.ndarray):
            np.save(OUT_DIR / f"{k}.npy", v)
        else:
            # bbx_cs list → CSV per element
            for i, arr in enumerate(v):
                np.savetxt(OUT_DIR / f"{k}_{i}.csv", arr, delimiter=",")
    print("📁  arrays saved to", OUT_DIR)


🗂  frames in file: 516

🔍  Frame 0
----------------------------------------
• patch_cam_t  shape=(1, 3)  dtype=float32
[[-0.5273324   0.26300016 45.6853    ]]

• poses  shape=(1, 46)  dtype=float32
[[-2.368138   -0.7633618  -1.7213504   0.80608803  0.20583363 -0.09330806  0.8838364   0.12469728 -0.06178701  0.11515429  0.7674963   0.04944598
   0.00572852  0.8486757   0.04685039 -0.14012277  0.08093543 -0.03927739 -0.5028792   0.00937823  0.00996734 -0.32637444  0.02291906 -0.01356386
   0.00067913  0.04569065  0.24808499  0.4783748  -0.42186233  0.7509638  -0.02185392  0.33443138  1.0059398   0.14834872 -0.03703762 -0.1092506
  -0.34541535 -0.5377565  -0.46241724 -0.69424284  0.01863243  0.38900214  1.075326    0.12092006 -0.10954121 -0.11959314]]

• betas  shape=(1, 10)  dtype=float32
[[-0.05001523  0.02085364 -0.07588243  0.10541424  0.00202027  0.06517917 -0.11400399 -0.02792621 -0.1094538  -0.07212697]]

• full_cam_t  shape=(1, 3)  dtype=float32
[[-3.4927106  1.4808805 20.028454 ]